In [1]:
import pandas as pd
import os, glob, re

In [2]:
import plotly.express as px
import matplotlib.pyplot as plt   # 맷폴렛- 그래프
%matplotlib inline

# ignore warnings
import warnings    # 워닝 방지
warnings.filterwarnings('ignore')

####### matplotlib - 한글 폰트 깨짐 방지 코드 #######
import platform

if platform.system() == 'Darwin':    # 맥
    font_name = 'AppleGothic'
elif platform.system() == 'Linux':   # 리눅스
    font_name = 'NanumGothic'
elif platform.system() == 'Windows': # 윈도우
    font_name = 'Malgun Gothic'
else:
    print('알수없는 시스템. 미적용')

plt.rc('font', family=font_name)

In [3]:
os.getcwd() # 현재 디렉토리위치

'C:\\studyNote\\Project'

## 데이터 로드

In [6]:
path = r'C:\data\\'
data_path = glob.glob(path+'배수시설.csv')
# 하나는 엑셀파일 하나는 csv 파일 -> 인것을 확인 
# 부산광역시_배수펌프장 및 유수지 현황_20200101_.xlsx >>> 이름 변경 -> 펌프장및유수지현황
# 부산시_배수시설_CSV.csv' >>> 이름 변경 -> 배수시설
data_path

['C:\\data\\\\배수시설.csv']

In [7]:
df_drainage = pd.read_csv(data_path[0], encoding='euc-kr') # utf-8 에러 -> euc-kr 사용
df_drainage.head()

,배수지명,착공일,준공일,시설용량\n(㎥/일),배수지규격(B m),배수지규격(L m),배수지규격(He m),평균체류시간\n(시간),배수지표고(E.L)\n(m),배수지\n배수방식,부지면적\n(㎡),유출관경\n(mm),고수위표고(E.L)\n(m),저수위표고(E.L)\n(m),배수지\n관리방식,사용여부,WGS84좌표(X),WGS84좌표(Y),위도좌표,경도좌표
0,감만,NaN,2002-12-01,"3,000",NaN,NaN,NaN,NaN,116.0,자연유하식,"5,777",400.0,114.5,110.0,NaN,NaN,NaN,NaN,35.119015,129.088007
1,감천,NaN,2002-12-30,"4,000",NaN,NaN,NaN,NaN,80.0,자연유하식,"4,777",500.0,84.0,79.0,NaN,NaN,NaN,NaN,35.078648,129.011243
2,개금1,NaN,1976-11-01,"15,000",NaN,NaN,NaN,14.0,70.0,자연유하식,"6,863",500.0,80.5,76.3,NaN,NaN,NaN,NaN,35.148822,129.022481
3,개금2,NaN,1969-09-01,"2,500",NaN,NaN,NaN,NaN,187.0,자연유하식,"8,163",300.0,186.1,181.6,NaN,NaN,NaN,NaN,35.142682,129.019421
4,거제,NaN,2001-12-29,"15,000",NaN,NaN,NaN,18.0,120.0,자연유하식,"11,329",700.0,131.0,125.0,NaN,NaN,NaN,NaN,35.184369,129.054550


# 결측값 확인 

In [8]:
print('데이터 길이 : ',len(df_drainage))

데이터 길이 :  74


In [9]:
col_name = list()
for idx,col in enumerate(df_drainage.columns):
    if df_drainage[col].isnull().sum() == 0: # 결측이 없는것을 제외하고 출력 
        pass
    elif df_drainage[col].isnull().sum() == 74:
        # 결측100% 인 컬럼 이름 리스트에 담기 
        col_name.append(df_drainage.columns[idx])
        print(f'결측 max_{df_drainage.columns[idx]} = ', df_drainage[col].isnull().sum())
    else:
        print(f'{df_drainage.columns[idx]} = ', df_drainage[col].isnull().sum())

결측 max_착공일 =  74
결측 max_배수지규격(B m) =  74
결측 max_배수지규격(L m) =  74
결측 max_배수지규격(He m) =  74
평균체류시간
(시간) =  36
배수지표고(E.L)
(m) =  10
배수지
배수방식 =  4
부지면적
(㎡) =  12
유출관경
(mm) =  24
고수위표고(E.L)
(m) =  10
저수위표고(E.L)
(m) =  10
결측 max_배수지
관리방식 =  74
결측 max_사용여부 =  74
결측 max_WGS84좌표(X) =  74
결측 max_WGS84좌표(Y) =  74


In [10]:
# 결측값이 전체인 아이들은 제거 결측은 데이터의 15%가 넘으면 사용하지 못함 -> 회의가 필요함 !  
for idx,col in enumerate(df_drainage.columns):
    percent = df_drainage[col].isnull().sum() / df_drainage[col].shape[0]
    if percent == 0.0: # 결측이 없는것을 제외하고 출력 
        pass
    elif percent <= 0.16:
        msg = 'column: {:>20}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (percent))
        print('0.16 아래 >>>>>', msg)
    else:
#         print(percent)
        msg = 'column: {:>20}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (percent))
        print(msg)

column:                  착공일	 Percent of NaN value: 100.00%
column:           배수지규격(B m)	 Percent of NaN value: 100.00%
column:           배수지규격(L m)	 Percent of NaN value: 100.00%
column:          배수지규격(He m)	 Percent of NaN value: 100.00%
column:          평균체류시간
(시간)	 Percent of NaN value: 48.65%
0.16 아래 >>>>> column:       배수지표고(E.L)
(m)	 Percent of NaN value: 13.51%
0.16 아래 >>>>> column:             배수지
배수방식	 Percent of NaN value: 5.41%
column:             부지면적
(㎡)	 Percent of NaN value: 16.22%
column:            유출관경
(mm)	 Percent of NaN value: 32.43%
0.16 아래 >>>>> column:       고수위표고(E.L)
(m)	 Percent of NaN value: 13.51%
0.16 아래 >>>>> column:       저수위표고(E.L)
(m)	 Percent of NaN value: 13.51%
column:             배수지
관리방식	 Percent of NaN value: 100.00%
column:                 사용여부	 Percent of NaN value: 100.00%
column:           WGS84좌표(X)	 Percent of NaN value: 100.00%
column:           WGS84좌표(Y)	 Percent of NaN value: 100.00%


# 컬럼 이름 표준화

In [11]:
col_name,len(col_name)

(['착공일',
  '배수지규격(B m)',
  '배수지규격(L m)',
  '배수지규격(He m)',
  '배수지\n관리방식',
  '사용여부',
  'WGS84좌표(X)',
  'WGS84좌표(Y)'],
 8)

In [12]:
le_col = [i for i in  df_drainage.columns if i not in col_name]
le_col, len(le_col)

(['배수지명',
  '준공일',
  '시설용량\n(㎥/일)',
  '평균체류시간\n(시간)',
  '배수지표고(E.L)\n(m)',
  '배수지\n배수방식',
  '부지면적\n(㎡)',
  '유출관경\n(mm)',
  '고수위표고(E.L)\n(m)',
  '저수위표고(E.L)\n(m)',
  '위도좌표',
  '경도좌표'],
 12)

In [13]:
# 결측 100 이랑 그외 네임 길이의 합 
len(col_name) + len(le_col)

20

In [14]:
# 총 컬럼 합
len(df_drainage.columns)

20

In [15]:
# 일단 100인거 먼저 사본 만들기
df_dra_notFull = df_drainage.loc[:,le_col] # 결측 100 아닌 데이터만 추출 
df_dra_notFull.head()

,배수지명,준공일,시설용량\n(㎥/일),평균체류시간\n(시간),배수지표고(E.L)\n(m),배수지\n배수방식,부지면적\n(㎡),유출관경\n(mm),고수위표고(E.L)\n(m),저수위표고(E.L)\n(m),위도좌표,경도좌표
0,감만,2002-12-01,"3,000",NaN,116.0,자연유하식,"5,777",400.0,114.5,110.0,35.119015,129.088007
1,감천,2002-12-30,"4,000",NaN,80.0,자연유하식,"4,777",500.0,84.0,79.0,35.078648,129.011243
2,개금1,1976-11-01,"15,000",14.0,70.0,자연유하식,"6,863",500.0,80.5,76.3,35.148822,129.022481
3,개금2,1969-09-01,"2,500",NaN,187.0,자연유하식,"8,163",300.0,186.1,181.6,35.142682,129.019421
4,거제,2001-12-29,"15,000",18.0,120.0,자연유하식,"11,329",700.0,131.0,125.0,35.184369,129.054550


# 사본 만들기 
### 필요없는 컬럼 생길시 사본에서 일차 처리하기   
->  준공일 , 그외 결측 과다 데이터, 근데 보기에는 다 의미있어보임 -> 회의가 필요 

In [16]:
df_dra_notFull.columns

Index(['배수지명', '준공일', '시설용량\n(㎥/일)', '평균체류시간\n(시간)', '배수지표고(E.L)\n(m)',
       '배수지\n배수방식', '부지면적\n(㎡)', '유출관경\n(mm)', '고수위표고(E.L)\n(m)',
       '저수위표고(E.L)\n(m)', '위도좌표', '경도좌표'],
      dtype='object')

In [17]:
# 컬럼 이름 변경 -> 만약에 이전에 필요 없는 컬럼 삭제시 해당 컬럼만 삭제 
change_name = {
    le_col[2]: '시설용량',
    le_col[3]: '평균체류시간',
    le_col[4]: '배수지표고',
    le_col[5]: '배수방식',
    le_col[6]: '부지면적',
    le_col[7]: '유출관크기',
    le_col[8]: '고수위',
    le_col[9]: '저수위',
    le_col[10]: '위도',
    le_col[11]: '경도',
}
change_name

{'시설용량\n(㎥/일)': '시설용량',
 '평균체류시간\n(시간)': '평균체류시간',
 '배수지표고(E.L)\n(m)': '배수지표고',
 '배수지\n배수방식': '배수방식',
 '부지면적\n(㎡)': '부지면적',
 '유출관경\n(mm)': '유출관크기',
 '고수위표고(E.L)\n(m)': '고수위',
 '저수위표고(E.L)\n(m)': '저수위',
 '위도좌표': '위도',
 '경도좌표': '경도'}

In [18]:
# 컬럼명 변경하기 
df_dra_notFull = df_dra_notFull.rename(change_name, axis='columns') # 변경확인 -> 원본 작업 진행
df_dra_notFull

,배수지명,준공일,시설용량,평균체류시간,배수지표고,배수방식,부지면적,유출관크기,고수위,저수위,위도,경도
0,감만,2002-12-01,"3,000",NaN,116.0,자연유하식,"5,777",400.0,114.5,110.0,35.119015,129.088007
1,감천,2002-12-30,"4,000",NaN,80.0,자연유하식,"4,777",500.0,84.0,79.0,35.078648,129.011243
2,개금1,1976-11-01,"15,000",14.0,70.0,자연유하식,"6,863",500.0,80.5,76.3,35.148822,129.022481
3,개금2,1969-09-01,"2,500",NaN,187.0,자연유하식,"8,163",300.0,186.1,181.6,35.142682,129.019421
4,거제,2001-12-29,"15,000",18.0,120.0,자연유하식,"11,329",700.0,131.0,125.0,35.184369,129.054550
...,...,...,...,...,...,...,...,...,...,...,...,...
69,수정1배수지,1932-11-01,"3,875",12.0,59.1,자연유하식,NaN,NaN,63.1,59.1,35.126047,129.040196
70,봉대산배수지,2016-09-30,"20,000",12.0,NaN,자연유하식,NaN,NaN,NaN,NaN,35.232081,129.223262
71,생곡공업배수지,2014-12-31,"4,400",12.0,NaN,자연유하식,NaN,NaN,NaN,NaN,35.135567,128.874763
72,괴정배수지,1978-03-31,"10,000",12.0,NaN,자연유하식,NaN,NaN,NaN,NaN,35.095763,128.997206


# 배수지명 표준화

In [19]:
df_dra_notFull['배수지명'].value_counts()

개금2      1
대티2      1
금곡       1
녹산       1
오륜       1
        ..
모라       1
문현       1
대티       1
구덕       1
월내배수지    1
Name: 배수지명, Length: 74, dtype: int64

In [20]:
# 속성의 마지막이 배수지로 끝나는 row만 추출 
df_dra_notFull[df_dra_notFull['배수지명'].str.endswith('배수지')]

,배수지명,준공일,시설용량,평균체류시간,배수지표고,배수방식,부지면적,유출관크기,고수위,저수위,위도,경도
63,반여2 배수지,2015-10-27,"12,000",NaN,173.5,병용식,"5,231",600.0,168.2,163.8,35.194507,129.132898
64,월내배수지,1981-01-01,500,NaN,NaN,자연유하식,0,NaN,NaN,NaN,35.329147,129.275430
65,생곡생활배수지,2014-12-01,"3,000",12.0,NaN,NaN,NaN,NaN,NaN,NaN,35.135556,128.874762
66,명례배수지,2014-12-01,"10,000",12.0,NaN,NaN,NaN,NaN,NaN,NaN,35.366376,129.252756
67,산성마을배수지,2014-12-01,250,12.0,NaN,NaN,NaN,NaN,NaN,NaN,35.239626,129.058812
68,장림배수지,1991-09-01,"10,000",12.0,NaN,NaN,NaN,NaN,NaN,NaN,35.067959,128.976526
69,수정1배수지,1932-11-01,"3,875",12.0,59.1,자연유하식,NaN,NaN,63.1,59.1,35.126047,129.040196
70,봉대산배수지,2016-09-30,"20,000",12.0,NaN,자연유하식,NaN,NaN,NaN,NaN,35.232081,129.223262
71,생곡공업배수지,2014-12-31,"4,400",12.0,NaN,자연유하식,NaN,NaN,NaN,NaN,35.135567,128.874763
72,괴정배수지,1978-03-31,"10,000",12.0,NaN,자연유하식,NaN,NaN,NaN,NaN,35.095763,128.997206


In [21]:
naming = list(df_dra_notFull['배수지명'])
# 맨뒤에 배수지라고 써있는 이름 표준화
re_row = [i[:-3] if i[-3:]  == '배수지' else i for i in naming]
# re_row, len(re_row)

In [22]:
# 배수지 새로운 이름으로 변경 
df_dra_notFull['배수지명'] = re_row

In [23]:
# 속성 변경확인 
df_dra_notFull[df_dra_notFull['배수지명'].str.endswith('배수지')]

,배수지명,준공일,시설용량,평균체류시간,배수지표고,배수방식,부지면적,유출관크기,고수위,저수위,위도,경도


#  중복데이터 확인 및 제거

In [24]:
df_dra_notFull['배수지명'].value_counts()
# 월내 데이터 -> 2 

월내      2
개금2     1
금곡      1
봉대산     1
오륜      1
       ..
문현      1
대티      1
구덕      1
대항      1
산성마을    1
Name: 배수지명, Length: 73, dtype: int64

In [25]:
# 중복값 제거 필요 : 64번 row가 데이터가 더 적은관계로 배제 
df_dra_notFull[df_dra_notFull['배수지명'] == '월내'] 

,배수지명,준공일,시설용량,평균체류시간,배수지표고,배수방식,부지면적,유출관크기,고수위,저수위,위도,경도
45,월내,1981-01-01,500,NaN,21.5,자연유하식,0,NaN,0.0,0.0,35.329147,129.27543
64,월내,1981-01-01,500,NaN,NaN,자연유하식,0,NaN,NaN,NaN,35.329147,129.27543


In [26]:
df_dra_notFull = df_dra_notFull.drop([64])

In [27]:
df_dra_notFull[df_dra_notFull['배수지명'] == '월내'] 

,배수지명,준공일,시설용량,평균체류시간,배수지표고,배수방식,부지면적,유출관크기,고수위,저수위,위도,경도
45,월내,1981-01-01,500,NaN,21.5,자연유하식,0,NaN,0.0,0.0,35.329147,129.27543


In [28]:
# 결측값이 전체인 아이들은 제거 결측은 데이터의 15%가 넘으면 사용하지 못함 -> 회의가 필요함 !  
for idx,col in enumerate(df_dra_notFull.columns):
    percent = df_dra_notFull[col].isnull().sum() / df_dra_notFull[col].shape[0]
    if percent == 0.0: # 결측이 없는것을 제외하고 출력 
        pass
    elif percent <= 0.16:
        msg = 'column: {:>20}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (percent))
        print('0.16 아래 >>>>>', msg)
    else:
#         print(percent)
        msg = 'column: {:>20}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (percent))
        print(msg)

column:               평균체류시간	 Percent of NaN value: 47.95%
0.16 아래 >>>>> column:                배수지표고	 Percent of NaN value: 12.33%
0.16 아래 >>>>> column:                 배수방식	 Percent of NaN value: 5.48%
column:                 부지면적	 Percent of NaN value: 16.44%
column:                유출관크기	 Percent of NaN value: 31.51%
0.16 아래 >>>>> column:                  고수위	 Percent of NaN value: 12.33%
0.16 아래 >>>>> column:                  저수위	 Percent of NaN value: 12.33%


In [29]:
# 인덱스 크기로 정렬 
df_dra_notFull.유출관크기.value_counts().sort_index()

100.0      1
150.0      1
200.0      3
250.0      4
300.0      8
350.0      1
400.0     10
500.0     10
600.0      3
700.0      6
800.0      1
1000.0     1
1200.0     1
Name: 유출관크기, dtype: int64

In [53]:
d = df_dra_notFull['유출관크기'].dropna()

In [56]:
# 유출관 크기 결측확인 -> 400.0으로 채우기
d.median()

400.0

In [52]:
df_dra_notFull

,배수지명,준공일,시설용량,평균체류시간,배수지표고,배수방식,부지면적,유출관크기,고수위,저수위,위도,경도
0,감만,2002-12-01,"3,000",NaN,116.0,자연유하식,"5,777",400.0,114.5,110.0,35.119015,129.088007
1,감천,2002-12-30,"4,000",NaN,80.0,자연유하식,"4,777",500.0,84.0,79.0,35.078648,129.011243
2,개금1,1976-11-01,"15,000",14.0,70.0,자연유하식,"6,863",500.0,80.5,76.3,35.148822,129.022481
3,개금2,1969-09-01,"2,500",NaN,187.0,자연유하식,"8,163",300.0,186.1,181.6,35.142682,129.019421
4,거제,2001-12-29,"15,000",18.0,120.0,자연유하식,"11,329",700.0,131.0,125.0,35.184369,129.054550
...,...,...,...,...,...,...,...,...,...,...,...,...
69,수정1,1932-11-01,"3,875",12.0,59.1,자연유하식,NaN,NaN,63.1,59.1,35.126047,129.040196
70,봉대산,2016-09-30,"20,000",12.0,NaN,자연유하식,NaN,NaN,NaN,NaN,35.232081,129.223262
71,생곡공업,2014-12-31,"4,400",12.0,NaN,자연유하식,NaN,NaN,NaN,NaN,35.135567,128.874763
72,괴정,1978-03-31,"10,000",12.0,NaN,자연유하식,NaN,NaN,NaN,NaN,35.095763,128.997206


In [ ]:
#유출관 크기 -> [ 100.0,  150.0,  200.0,  250.0,  300.0,  350.0,  400.0,  500.0,
#                600.0,  700.0,  800.0, 1000.0, 1200.0]

In [ ]:
df = df_dra_notFull.iloc[:,[0,2,4,5,6,7,8,9,10,11]]

In [ ]:
df[df.유출관크기 == 1200.0] # 동래구 명장 배수지 

In [ ]:
df[df.유출관크기 == 1000.0] # 동래구 명장 배수지 

In [ ]:
df[df.유출관크기 == 100.0]

In [ ]:
df[df.유출관크기 == 150.0]

In [ ]:
df.head(1)

In [ ]:
df_dra_notFull[df_dra_notFull['배수지명'].str.startswith('수정')]

In [ ]:
# pip install plotly
import plotly.express as px 

fig = px.scatter_mapbox(df, lat="위도", lon="경도", hover_name="배수지명", hover_data=["부지면적", "배수방식"],
                        color_discrete_sequence=["#FF6692"], zoom=11, height=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# 데이터 만들기 

In [37]:
# df.to_csv(path+'\배수지데이터.csv', sep=',', na_rep='NaN',encoding='utf-8-sig')

# 배수 두번째 파일 작업 

여기서부터 위와 같이 진행 

[환경](http://www.busan.go.kr/environment/ahriver01)

혹시 몰라서 링크 올려둠 도움이 될지는 모르겠음 <br>
<br>
[참고자료-1](https://github.com/Jerrykim91/studyNote/blob/master/Project/%EA%B0%95%EC%88%98%EB%9F%89_%EC%A0%84%EC%B2%98%EB%A6%AC_ver1.ipynb)<br>
[참고자료-번외1](https://github.com/Jerrykim91/studyNote/blob/master/Project/Idea_check_2020.09.13.ipynb)<br>
[참고자료-번외2](https://github.com/Jerrykim91/C19/blob/master/C19_PatientInfo.ipynb)<br>
[참고자료-번외3-시각화(matplotlib)](https://github.com/Jerrykim91/Study_Through_Kaggle/blob/master/Pandas/Pandas04.ipynb)<br>
[참고자료-번외4-시각화(Plotly)](https://dailyheumsi.tistory.com/118)

In [38]:
import pandas as pd
import os, glob, re

import plotly.express as px
import matplotlib.pyplot as plt   # 맷폴렛- 그래프
%matplotlib inline

import plotly.express as px 

# ignore warnings
import warnings    # 워닝 방지
warnings.filterwarnings('ignore')

####### matplotlib - 한글 폰트 깨짐 방지 코드 #######
import platform

if platform.system() == 'Darwin':    # 맥
    font_name = 'AppleGothic'
elif platform.system() == 'Linux':   # 리눅스
    font_name = 'NanumGothic'
elif platform.system() == 'Windows': # 윈도우
    font_name = 'Malgun Gothic'
else:
    print('알수없는 시스템. 미적용')

plt.rc('font', family=font_name)

os.getcwd() # 현재 디렉토리위치


'C:\\studyNote\\Project'

In [39]:
path = r'C:\data\\'
data_path = glob.glob(path+'펌프장및유수지현황.xlsx')
# 하나는 엑셀파일 하나는 csv 파일 -> 인것을 확인 
# 부산광역시_배수펌프장 및 유수지 현황_20200101_.xlsx >>> 이름 변경 -> 펌프장및유수지현황
# 부산시_배수시설_CSV.csv' >>> 이름 변경 -> 배수시설
data_path

['C:\\data\\\\펌프장및유수지현황.xlsx']

In [40]:
df_pump     = pd.read_excel(data_path[0], encoding='euc-kr')# utf-8 에러 -> euc-kr 사용
df_pump.head(2)

,구군,배수펌프장명,위치,Latitude,Longitude,펌프시설 전동기(HP),펌프시설 엔진(HP),펌프시설 설치년도,설계빈도(년),배수량(㎥/분),비상발전기 시설,비상발전기 시설년도,유수지 면적(㎡),유수지 용량(㎡),비고
0,동래구,명륜,온천천로 203,35.20162,129.079404,"400×1(A), 400×1(B), 400×1(C), 400×1(D)",-,"2012(A), 2014(B), 2016(C), 2017(D)",30,784.0,"1250KW×1, 600KW×2","2018, 2011",400,1200,내수배제
1,동래구,수안,명륜로 70,NaN,NaN,46×2,-,2010,30,60.0,한전2회선,-,-,-,내수배제


In [41]:
df_pump.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   구군            59 non-null     object 
 1   배수펌프장명        62 non-null     object 
 2   위치            60 non-null     object 
 3   Latitude      45 non-null     float64
 4   Longitude     45 non-null     float64
 5   펌프시설 전동기(HP)  62 non-null     object 
 6   펌프시설 엔진(HP)   61 non-null     object 
 7   펌프시설 설치년도     61 non-null     object 
 8   설계빈도(년)       61 non-null     object 
 9   배수량(㎥/분)      61 non-null     float64
 10  비상발전기 시설      59 non-null     object 
 11  비상발전기 시설년도    54 non-null     object 
 12  유수지 면적(㎡)     58 non-null     object 
 13  유수지 용량(㎡)     58 non-null     object 
 14  비고            58 non-null     object 
dtypes: float64(3), object(12)
memory usage: 7.4+ KB


In [42]:
tmp = list(df_pump.columns)
tmp
# 제외할 데이허 혹은 살릴 데이터 선정  ->>>>>  이거는 유수지 데이터인듯 

['구군',
 '배수펌프장명',
 '위치',
 'Latitude',
 'Longitude',
 '펌프시설 전동기(HP)',
 '펌프시설 엔진(HP)',
 '펌프시설 설치년도',
 '설계빈도(년)',
 '배수량(㎥/분)',
 '비상발전기 시설',
 '비상발전기 시설년도',
 '유수지 면적(㎡)',
 '유수지 용량(㎡)',
 '비고']

In [43]:
col_name = list()
for idx,col in enumerate(df_pump.columns):
    if df_pump[col].isnull().sum() == 0: # 결측이 없는것을 제외하고 출력 
        pass
    elif df_pump[col].isnull().sum() == 74:
        # 결측100% 인 컬럼 이름 리스트에 담기 
        col_name.append(df_pump.columns[idx])
        print(f'결측 max_{df_pump.columns[idx]} = ', df_pump[col].isnull().sum())
    else:
        print(f'{df_pump.columns[idx]} = ', df_pump[col].isnull().sum())

구군 =  3
위치 =  2
Latitude =  17
Longitude =  17
펌프시설 엔진(HP) =  1
펌프시설 설치년도 =  1
설계빈도(년) =  1
배수량(㎥/분) =  1
비상발전기 시설 =  3
비상발전기 시설년도 =  8
유수지 면적(㎡) =  4
유수지 용량(㎡) =  4
비고 =  4


In [44]:
# 결측값이 전체인 아이들은 제거 결측은 데이터의 15%가 넘으면 사용하지 못함 -> 회의가 필요함 !  
for idx,col in enumerate(df_pump.columns):
    percent = df_pump[col].isnull().sum() / df_pump[col].shape[0]
    if percent == 0.0: # 결측이 없는것을 제외하고 출력 
        pass
    elif percent <= 0.16:
        msg = 'column: {:>20}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (percent))
#         print('0.16 아래 >>>>>', msg)
    else:
#         print(percent)
        msg = 'column: {:>20}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (percent))
        print('결측값 과다 ',msg)

결측값 과다  column:             Latitude	 Percent of NaN value: 27.42%
결측값 과다  column:            Longitude	 Percent of NaN value: 27.42%


In [45]:
print('데이터 길이 : ',len(df_pump))

데이터 길이 :  62


In [46]:
## 해야하는 것 : 컬럼이름 표준화 
## 구군, 배수장 이름 확인 
print('<<<< 1번 배수장명 데이터 >>>> \n', re_row) 

<<<< 1번 배수장명 데이터 >>>> 
 ['감만', '감천', '개금1', '개금2', '거제', '고촌', '구덕', '구포2', '금곡', '꽃마을', '남부민', '남산', '녹산', '녹산공업용수', '당감', '대청', '대티', '대티2', '대항', '만덕1', '만덕2', '만덕5', '망미', '명장', '모라', '문현', '미음공업', '미음생활', '반송', '반여', '범일', '복병산', '부산신항', '서동', '수정2', '수정산터널', '신평', '아미', '안창', '엄궁', '연산', '영도', '영주', '오륜', '용호2', '월내', '장안', '전포1', '전포2', '정관산단', '정관택지고지', '정관택지저지', '좌동1', '좌동2', '주례1', '주례2', '중동', '지사', '지사공업', '천성', '청학', '해돋이', '화전산단', '반여2 ', '월내', '생곡생활', '명례', '산성마을', '장림', '수정1', '봉대산', '생곡공업', '괴정', '용호1']


In [47]:
naming = list(df_pump['배수펌프장명'])

In [48]:
df_pump['배수펌프장명'].value_counts()

송정          2
거제          2
엄궁3         1
장안          1
범방          1
명지          1
춘천          1
본녹산         1
동천1         1
하신          1
맥도          1
송정1게이트      1
송정2게이트      1
신포          1
연산9동        1
명지동진        1
송선마을        1
감전3         1
금사          1
엄궁1         1
동천2         1
감전2         1
자성대1        1
반여1         1
방근          1
상덕          1
감전1         1
녹산          1
연산4동        1
반여2         1
신금로         1
길천          1
학장          1
낙민          1
하단2         1
구남          1
수안          1
작지          1
덕천          1
자성대2        1
감전4         1
감전천         1
식만          1
구포          1
하단1         1
반여3         1
삼성시장        1
사덕          1
성산마을        1
장안(간이펌프)    1
엄궁2         1
명륜          1
대저          1
장림1         1
장림2         1
용호          1
중덕          1
서금사         1
동방          1
성산          1
Name: 배수펌프장명, dtype: int64

In [49]:
# 중복 row 확인 - 송정 
df_pump[df_pump['배수펌프장명'] == '송정']

,구군,배수펌프장명,위치,Latitude,Longitude,펌프시설 전동기(HP),펌프시설 엔진(HP),펌프시설 설치년도,설계빈도(년),배수량(㎥/분),비상발전기 시설,비상발전기 시설년도,유수지 면적(㎡),유수지 용량(㎡),비고
19,해운대구,송정,해운대로 1100,35.183349,129.201155,100×4,-,2012,50,120.0,한전2회선,-,600,1800,내수배제
34,강서구,송정,가락대로 227,NaN,NaN,320×6,-,1999,30,1500.0,"한전2회선, 300㎾×3",1999,15000,15500,하천


In [50]:
# 중복 row 확인 - 거제
df_pump[df_pump['배수펌프장명'] == '거제']

,구군,배수펌프장명,위치,Latitude,Longitude,펌프시설 전동기(HP),펌프시설 엔진(HP),펌프시설 설치년도,설계빈도(년),배수량(㎥/분),비상발전기 시설,비상발전기 시설년도,유수지 면적(㎡),유수지 용량(㎡),비고
25,연제구,거제,반송로 107,35.238782,128.668938,580×9,-,2017,30,3960.0,한전2회선,-,-,-,내수배제
26,NaN,거제,거제천로269번길41,NaN,NaN,580×2,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# 1번 데이터와 2번데이터 겹치는 데이터 확인 -> 전혀다른 데이터임
for i in naming:
    if i in re_row:
        print(i)

거제
거제
장안
녹산


In [52]:
# 중복되는 데이터 확인 
df_pump[df_pump['배수펌프장명'] == '거제']

,구군,배수펌프장명,위치,Latitude,Longitude,펌프시설 전동기(HP),펌프시설 엔진(HP),펌프시설 설치년도,설계빈도(년),배수량(㎥/분),비상발전기 시설,비상발전기 시설년도,유수지 면적(㎡),유수지 용량(㎡),비고
25,연제구,거제,반송로 107,35.238782,128.668938,580×9,-,2017,30,3960.0,한전2회선,-,-,-,내수배제
26,NaN,거제,거제천로269번길41,NaN,NaN,580×2,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df_dra_notFull[df_dra_notFull['배수지명'] == '거제'] 

,배수지명,준공일,시설용량,평균체류시간,배수지표고,배수방식,부지면적,유출관크기,고수위,저수위,위도,경도
4,거제,2001-12-29,"15,000",18.0,120.0,자연유하식,"11,329",700.0,131.0,125.0,35.184369,129.05455


In [54]:
df_pump[df_pump['배수펌프장명'] == '장안']

,구군,배수펌프장명,위치,Latitude,Longitude,펌프시설 전동기(HP),펌프시설 엔진(HP),펌프시설 설치년도,설계빈도(년),배수량(㎥/분),비상발전기 시설,비상발전기 시설년도,유수지 면적(㎡),유수지 용량(㎡),비고
32,기장군,장안,좌천리 241-2,35.311966,129.245999,215×4,-,2016,30,540.0,900㎾×1,2016,2200,4840,내수배제


In [55]:
df_dra_notFull[df_dra_notFull['배수지명'] == '장안'] 

,배수지명,준공일,시설용량,평균체류시간,배수지표고,배수방식,부지면적,유출관크기,고수위,저수위,위도,경도
46,장안,2010-02-02,"2,000",NaN,94.5,자연유하식,0,300.0,97.1,93.1,35.323601,129.259274


In [56]:
df_pump[df_pump['배수펌프장명'] == '녹산']

,구군,배수펌프장명,위치,Latitude,Longitude,펌프시설 전동기(HP),펌프시설 엔진(HP),펌프시설 설치년도,설계빈도(년),배수량(㎥/분),비상발전기 시설,비상발전기 시설년도,유수지 면적(㎡),유수지 용량(㎡),비고
41,강서구,녹산,낙동남로 791,35.12031,128.897318,"1200×19, 738×1, 497×1",-,2009,200,12000.0,"한전2회선, 1750㎾×3",2009,-,-,하천


In [57]:
df_dra_notFull[df_dra_notFull['배수지명'] == '녹산'] 

,배수지명,준공일,시설용량,평균체류시간,배수지표고,배수방식,부지면적,유출관크기,고수위,저수위,위도,경도
12,녹산,1999-09-01,"17,000",28.0,0.0,자연유하식,0,NaN,0.0,0.0,35.099451,128.809761


In [58]:
tmp = list(df_pump.columns)
print(tmp)
# 제외할 데이허 혹은 살릴 데이터 선정  ->>>>>  이거는 유수지 데이터인듯 
# 구군, 배수펌프장명,위치,배수량,'유수지 면적(㎡)', '유수지 용량(㎡)' 비고 

['구군', '배수펌프장명', '위치', 'Latitude', 'Longitude', '펌프시설 전동기(HP)', '펌프시설 엔진(HP)', '펌프시설 설치년도', '설계빈도(년)', '배수량(㎥/분)', '비상발전기 시설', '비상발전기 시설년도', '유수지 면적(㎡)', '유수지 용량(㎡)', '비고']


In [59]:
re_col = ['구군', '배수펌프장명', '위치', '배수량(㎥/분)', '유수지 면적(㎡)', '유수지 용량(㎡)', '비고']
# re_col = ['시구군', '배수펌프장명', '위치', '배수량', '유수지 면적', '유수지 용량', '비고']
re_col

['구군', '배수펌프장명', '위치', '배수량(㎥/분)', '유수지 면적(㎡)', '유수지 용량(㎡)', '비고']

In [60]:
# 결측 100 아닌 데이터만 추출 
df_pump_re = df_pump.loc[:,re_col] 
df_pump_re

,구군,배수펌프장명,위치,배수량(㎥/분),유수지 면적(㎡),유수지 용량(㎡),비고
0,동래구,명륜,온천천로 203,784.0,400,1200,내수배제
1,동래구,수안,명륜로 70,60.0,-,-,내수배제
2,동래구,낙민,온천천로 339번길28,40.0,-,-,내수배제
3,동구,자성대1,자성로140번길33,140.0,-,-,내수배제
4,동구,자성대2,자성로140번길36,300.0,-,-,내수배제
...,...,...,...,...,...,...,...
57,사상구,엄궁1,강변대로 532번길 71,940.0,41089(엄궁1·엄궁2·엄궁3),90396(엄궁1·엄궁2·엄궁3),하천
58,사상구,엄궁2,NaN,180.0,NaN,NaN,NaN
59,사상구,엄궁3,NaN,1700.0,NaN,NaN,NaN
60,사상구,학장,농산물시장로 25번길 52-12,285.0,7440,16368,내수배제


In [61]:
# 컬럼 이름 변경 -> 만약에 이전에 필요 없는 컬럼 삭제시 해당 컬럼만 삭제 
change_na = {
    tmp[0]: '시구군',
    tmp[1]: '배수펌프장명',
    tmp[2]: '위치',
    tmp[9]: '배수량',
    tmp[-3]: '유수지 면적',
    tmp[-2]: '유수지 용량',
    tmp[-1]: '배수방식',
}
change_na

{'구군': '시구군',
 '배수펌프장명': '배수펌프장명',
 '위치': '위치',
 '배수량(㎥/분)': '배수량',
 '유수지 면적(㎡)': '유수지 면적',
 '유수지 용량(㎡)': '유수지 용량',
 '비고': '배수방식'}

In [62]:
df_pump_re = df_pump_re.rename(change_na, axis='columns')
df_pump_re = df_pump_re.replace('-','NaN')
df_pump_re

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
0,동래구,명륜,온천천로 203,784.0,400,1200,내수배제
1,동래구,수안,명륜로 70,60.0,NaN,NaN,내수배제
2,동래구,낙민,온천천로 339번길28,40.0,NaN,NaN,내수배제
3,동구,자성대1,자성로140번길33,140.0,NaN,NaN,내수배제
4,동구,자성대2,자성로140번길36,300.0,NaN,NaN,내수배제
...,...,...,...,...,...,...,...
57,사상구,엄궁1,강변대로 532번길 71,940.0,41089(엄궁1·엄궁2·엄궁3),90396(엄궁1·엄궁2·엄궁3),하천
58,사상구,엄궁2,NaN,180.0,NaN,NaN,NaN
59,사상구,엄궁3,NaN,1700.0,NaN,NaN,NaN
60,사상구,학장,농산물시장로 25번길 52-12,285.0,7440,16368,내수배제


In [63]:
df_pump_re['유수지 면적'].value_counts()

NaN                   17
38                     2
2200                   1
7030                   1
3900                   1
1246                   1
1500                   1
4800                   1
15000                  1
220                    1
7440                   1
4500                   1
865                    1
1233                   1
400                    1
10                     1
840                    1
8515                   1
322                    1
1750                   1
32                     1
80182(감전1·감전2)         1
483                    1
49                     1
6268                   1
11600                  1
760                    1
2600                   1
310                    1
500                    1
1010                   1
3000                   1
41089(엄궁1·엄궁2·엄궁3)     1
600                    1
109                    1
56500                  1
2236                   1
8000                   1
232                    1
246                    1


# 유수지 면적중 이상치 
## 엄궁 -> 3군대 합! / 감전 -> 2군대 유수지 합 

In [64]:
df_pump_re[df_pump_re['유수지 면적'] == '41089(엄궁1·엄궁2·엄궁3)']

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
57,사상구,엄궁1,강변대로 532번길 71,940.0,41089(엄궁1·엄궁2·엄궁3),90396(엄궁1·엄궁2·엄궁3),하천


In [65]:
df_pump_re[df_pump_re['유수지 용량'] == '90396(엄궁1·엄궁2·엄궁3)']

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
57,사상구,엄궁1,강변대로 532번길 71,940.0,41089(엄궁1·엄궁2·엄궁3),90396(엄궁1·엄궁2·엄궁3),하천


In [66]:
df_pump_re = df_pump_re.replace('41089(엄궁1·엄궁2·엄궁3)','41089')
df_pump_re = df_pump_re.replace('90396(엄궁1·엄궁2·엄궁3)','90396')
df_pump_re['유수지 면적'].value_counts()

NaN               17
38                 2
2200               1
4500               1
1246               1
1500               1
4800               1
15000              1
220                1
7440               1
7030               1
1233               1
32                 1
400                1
41089              1
10                 1
840                1
8515               1
322                1
1750               1
3900               1
483                1
865                1
3000               1
11600              1
760                1
2600               1
310                1
500                1
1010               1
49                 1
600                1
6268               1
109                1
56500              1
2236               1
8000               1
232                1
246                1
80182(감전1·감전2)     1
128                1
Name: 유수지 면적, dtype: int64

In [67]:
df_pump_re[df_pump_re['유수지 면적'] == '80182(감전1·감전2)']

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
53,사상구,감전1,낙동로 1097,1620.0,80182(감전1·감전2),176400(감전1·감전2),하천


In [68]:
df_pump_re[df_pump_re['유수지 용량'] == '176400(감전1·감전2)']

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
53,사상구,감전1,낙동로 1097,1620.0,80182(감전1·감전2),176400(감전1·감전2),하천


In [69]:
df_pump_re = df_pump_re.replace('80182(감전1·감전2)','80182')
df_pump_re = df_pump_re.replace('176400(감전1·감전2)','176400')
df_pump_re['유수지 면적'].value_counts()

NaN      17
38        2
2200      1
4500      1
1246      1
1500      1
4800      1
15000     1
220       1
7440      1
7030      1
1233      1
32        1
400       1
41089     1
10        1
840       1
8515      1
322       1
1750      1
3900      1
483       1
865       1
49        1
11600     1
80182     1
760       1
2600      1
310       1
500       1
1010      1
3000      1
6268      1
600       1
109       1
56500     1
2236      1
8000      1
232       1
246       1
128       1
Name: 유수지 면적, dtype: int64

In [70]:
df_pump_re.배수방식.value_counts()

내수배제    49
하천       7
NaN      2
Name: 배수방식, dtype: int64

In [71]:
df_pump_re[df_pump_re.배수방식 == 'NaN']

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
30,NaN,서금사,금사동 545-9,580.0,NaN,NaN,NaN
31,기장군,길천,길천리 190-36,9.0,38,96,NaN


In [72]:
# NaN를 금정구로 변경
df_pump_re.시구군[30] = '금정구'

In [73]:
df_pump_re[df_pump_re.배수방식 == 'NaN']

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
30,금정구,서금사,금사동 545-9,580.0,NaN,NaN,NaN
31,기장군,길천,길천리 190-36,9.0,38,96,NaN


In [74]:
df_pump_re.위치[30] = '금사동 545-9'

In [75]:
df_pump_re[df_pump_re.배수방식 == 'NaN']

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
30,금정구,서금사,금사동 545-9,580.0,NaN,NaN,NaN
31,기장군,길천,길천리 190-36,9.0,38,96,NaN


In [76]:
df_pump_re

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
0,동래구,명륜,온천천로 203,784.0,400,1200,내수배제
1,동래구,수안,명륜로 70,60.0,NaN,NaN,내수배제
2,동래구,낙민,온천천로 339번길28,40.0,NaN,NaN,내수배제
3,동구,자성대1,자성로140번길33,140.0,NaN,NaN,내수배제
4,동구,자성대2,자성로140번길36,300.0,NaN,NaN,내수배제
...,...,...,...,...,...,...,...
57,사상구,엄궁1,강변대로 532번길 71,940.0,41089,90396,하천
58,사상구,엄궁2,NaN,180.0,NaN,NaN,NaN
59,사상구,엄궁3,NaN,1700.0,NaN,NaN,NaN
60,사상구,학장,농산물시장로 25번길 52-12,285.0,7440,16368,내수배제


In [77]:
df_pump_re.위치.isnull().sum()

2

In [78]:
len(df_pump_re.위치.value_counts())

60

In [79]:
len(df_pump_re)

62

In [80]:
df_pump_re[df_pump_re.시구군 == '사상구']

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
53,사상구,감전1,낙동로 1097,1620.0,80182,176400,하천
54,사상구,감전2,낙동로 1142,2640.0,NaN,NaN,NaN
55,사상구,감전3,낙동대로 952 맞은편,50.0,49,273,내수배제
56,사상구,감전4,낙동대로 1024 맞은편,90.0,322,590,내수배제
57,사상구,엄궁1,강변대로 532번길 71,940.0,41089,90396,하천
58,사상구,엄궁2,NaN,180.0,NaN,NaN,NaN
59,사상구,엄궁3,NaN,1700.0,NaN,NaN,NaN
60,사상구,학장,농산물시장로 25번길 52-12,285.0,7440,16368,내수배제
61,사상구,감전천,낙동대로 1016번길 60,1370.0,1750,6500,하천


## 엄궁동 유수지 데이터 -> 널값 채우기 

In [81]:
df_pump_re.위치[58] = '강변대로 532번길 71'
df_pump_re.위치[59] = '강변대로 532번길 71'
df_pump_re.배수방식[58] = '하천'
df_pump_re.배수방식[59] = '하천'
df_pump_re['유수지 면적'][58] = '41089'
df_pump_re['유수지 면적'][59] = '41089'
df_pump_re['유수지 용량'][58] = '90396'
df_pump_re['유수지 용량'][59] = '90396'

In [82]:
df_pump_re[df_pump_re.시구군 == '사상구']

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
53,사상구,감전1,낙동로 1097,1620.0,80182,176400,하천
54,사상구,감전2,낙동로 1142,2640.0,NaN,NaN,NaN
55,사상구,감전3,낙동대로 952 맞은편,50.0,49,273,내수배제
56,사상구,감전4,낙동대로 1024 맞은편,90.0,322,590,내수배제
57,사상구,엄궁1,강변대로 532번길 71,940.0,41089,90396,하천
58,사상구,엄궁2,강변대로 532번길 71,180.0,41089,90396,하천
59,사상구,엄궁3,강변대로 532번길 71,1700.0,41089,90396,하천
60,사상구,학장,농산물시장로 25번길 52-12,285.0,7440,16368,내수배제
61,사상구,감전천,낙동대로 1016번길 60,1370.0,1750,6500,하천


## 감전동 유수지 데이터 -> 널값 채우기 

In [83]:
df_pump_re[df_pump_re['배수펌프장명'].str.startswith('감전')]

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
53,사상구,감전1,낙동로 1097,1620.0,80182,176400,하천
54,사상구,감전2,낙동로 1142,2640.0,NaN,NaN,NaN
55,사상구,감전3,낙동대로 952 맞은편,50.0,49,273,내수배제
56,사상구,감전4,낙동대로 1024 맞은편,90.0,322,590,내수배제
61,사상구,감전천,낙동대로 1016번길 60,1370.0,1750,6500,하천


In [84]:
# 잠전 2동은 1동과 같은위치 같은 용량으로 공지되어있음 
df_pump_re['유수지 면적'][54] = '80182'
df_pump_re['유수지 용량'][54] = '176400'
df_pump_re.배수방식[54] = '하천'

In [85]:
df_pump_re[df_pump_re['배수펌프장명'].str.startswith('감전')]

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
53,사상구,감전1,낙동로 1097,1620.0,80182,176400,하천
54,사상구,감전2,낙동로 1142,2640.0,80182,176400,하천
55,사상구,감전3,낙동대로 952 맞은편,50.0,49,273,내수배제
56,사상구,감전4,낙동대로 1024 맞은편,90.0,322,590,내수배제
61,사상구,감전천,낙동대로 1016번길 60,1370.0,1750,6500,하천


In [86]:
df_pump_re

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
0,동래구,명륜,온천천로 203,784.0,400,1200,내수배제
1,동래구,수안,명륜로 70,60.0,NaN,NaN,내수배제
2,동래구,낙민,온천천로 339번길28,40.0,NaN,NaN,내수배제
3,동구,자성대1,자성로140번길33,140.0,NaN,NaN,내수배제
4,동구,자성대2,자성로140번길36,300.0,NaN,NaN,내수배제
...,...,...,...,...,...,...,...
57,사상구,엄궁1,강변대로 532번길 71,940.0,41089,90396,하천
58,사상구,엄궁2,강변대로 532번길 71,180.0,41089,90396,하천
59,사상구,엄궁3,강변대로 532번길 71,1700.0,41089,90396,하천
60,사상구,학장,농산물시장로 25번길 52-12,285.0,7440,16368,내수배제


In [87]:
df_pump_re[df_pump_re.시구군 == '동구']

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
3,동구,자성대1,자성로140번길33,140.0,NaN,NaN,내수배제
4,동구,자성대2,자성로140번길36,300.0,NaN,NaN,내수배제


In [88]:
df_pump_re.시구군.value_counts()

강서구     19
사상구      9
해운대구     7
남구       5
사하구      4
연제구      4
기장군      3
북구       3
동래구      3
동구       2
금정구      1
Name: 시구군, dtype: int64

In [89]:
df_pump_re[df_pump_re.시구군 == '해운대구']

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
13,해운대구,춘천,해운대 해변로 209 번가길32,1070.0,1246,6000,내수배제
14,해운대구,반여1,수영강변대로 353,100.0,500,1700,내수배제
15,해운대구,반여2,수영강변대로 606,1460.0,1010,8500,내수배제
16,해운대구,반여3,선수촌로 119,44.0,38,75,내수배제
17,해운대구,송정1게이트,송정강변로 51,120.0,NaN,NaN,내수배제
18,해운대구,송정2게이트,송정중앙로 15번길 120-3,120.0,NaN,NaN,내수배제
19,해운대구,송정,해운대로 1100,120.0,600,1800,내수배제


In [90]:
df_pump_re.위치.value_counts().keys()

Index(['강변대로 532번길 71', '금사동 545-9', '좌천리 242-28', '명지오션시티 1로 328',
       '낙동대로 1024 맞은편', '생곡로 56-1', '이기대공원로 7', '구포만세길 10', '낙동대로 1016번길 60',
       '온천천남로39옆', '을숙도대로 480번길40', '영강길 20', '낙동로 1142', '좌천리 241-2',
       '수영강변대로 353', '녹산산업 북로221번길 17', '공항로 393번길 4', '낙동대로 952 맞은편',
       '신선로 514', '낙동대로 1769', '전포대로 91번길47', '자성로140번길33', '경전철로 109',
       '수영강변대로 606', '낙동남로 791', '용호동 261-5일원', '낙동로 1097', '금사동545-19',
       '반송로 107', '제도로 1041번길 206', '거제천로269번길41', '온천천로 203',
       '송정중앙로 15번길 120-3', '하신중앙로53번길65', '공항로 533번길 1', '명지국제11로85',
       '범방동 341', '명지오션시티 13로 5', '중앙대로1164앞', '하신번영로243', '온천천로 339번길28',
       '하단동 1163일원', '남동천로 8-2', '송정강변로 51', '길천리 190-36', '낙동대로 1739번길16',
       '온천천남로110 (한양아파트)앞', '농산물시장로 25번길 52-12', '유통단지 1로7', '화전산업대로 232',
       '해운대로 1100', '해운대 해변로 209 번가길32', '상덕로 49번길 47-38', '명륜로 70',
       '자성로140번길36', '대저로 274번길 28', '식만로 131', '선수촌로 119', '가락대로 227',
       '녹산동 49-1'],
      dtype='object')

In [91]:
# df_pump_re.to_csv(path+'\유수지현황.csv', sep=',', na_rep='NaN',encoding='utf-8-sig')

## 유수지위경도 추가하고 위치데이터 변경

In [92]:
# 폴더 안에 데이터 확인 
glob.glob(path+'*')

['C:\\data\\srcdata',
 'C:\\data\\데이터',
 'C:\\data\\배수시설.csv',
 'C:\\data\\배수지데이터.csv',
 'C:\\data\\유수지위경도.csv',
 'C:\\data\\유수지현황.csv',
 'C:\\data\\펌프장및유수지현황.xlsx']

In [93]:
path = r'C:\data\\'
data_path = glob.glob(path+'유수지위경도.csv')
data_path

['C:\\data\\\\유수지위경도.csv']

In [94]:
df_Location = pd.read_csv(data_path[0], encoding='euc-kr') # utf-8 에러 -> euc-kr 사용
df_Location.head()

,위치,Latitude,Longitude
0,온천천로 203,35.201620,129.079404
1,명륜로 70,NaN,NaN
2,온천천로 339번길28,35.195371,129.091783
3,자성로140번길33,NaN,NaN
4,자성로140번길36,NaN,NaN


In [95]:
df_Location.위치

0              온천천로 203
1                명륜로 70
2          온천천로 339번길28
3            자성로140번길33
4            자성로140번길36
            ...        
57        강변대로 532번길 71
58        강변대로 532번길 71
59        강변대로 532번길 71
60    농산물시장로 25번길 52-12
61       낙동대로 1016번길 60
Name: 위치, Length: 62, dtype: object

In [96]:
df_pump_re

,시구군,배수펌프장명,위치,배수량,유수지 면적,유수지 용량,배수방식
0,동래구,명륜,온천천로 203,784.0,400,1200,내수배제
1,동래구,수안,명륜로 70,60.0,NaN,NaN,내수배제
2,동래구,낙민,온천천로 339번길28,40.0,NaN,NaN,내수배제
3,동구,자성대1,자성로140번길33,140.0,NaN,NaN,내수배제
4,동구,자성대2,자성로140번길36,300.0,NaN,NaN,내수배제
...,...,...,...,...,...,...,...
57,사상구,엄궁1,강변대로 532번길 71,940.0,41089,90396,하천
58,사상구,엄궁2,강변대로 532번길 71,180.0,41089,90396,하천
59,사상구,엄궁3,강변대로 532번길 71,1700.0,41089,90396,하천
60,사상구,학장,농산물시장로 25번길 52-12,285.0,7440,16368,내수배제


In [97]:
df_pump_re.위치 = df_Location.위치

In [98]:
df_pump_re['위도'] = df_Location.Latitude
df_pump_re['경도'] = df_Location.Longitude

In [99]:
df_pump_re.columns

Index(['시구군', '배수펌프장명', '위치', '배수량', '유수지 면적', '유수지 용량', '배수방식', '위도', '경도'], dtype='object')

In [100]:
df_pump_re = df_pump_re[['시구군', '배수펌프장명', '배수량', '유수지 면적', '유수지 용량', '배수방식','위치', '위도', '경도']]

In [101]:
df_pump_re

,시구군,배수펌프장명,배수량,유수지 면적,유수지 용량,배수방식,위치,위도,경도
0,동래구,명륜,784.0,400,1200,내수배제,온천천로 203,35.201620,129.079404
1,동래구,수안,60.0,NaN,NaN,내수배제,명륜로 70,NaN,NaN
2,동래구,낙민,40.0,NaN,NaN,내수배제,온천천로 339번길28,35.195371,129.091783
3,동구,자성대1,140.0,NaN,NaN,내수배제,자성로140번길33,NaN,NaN
4,동구,자성대2,300.0,NaN,NaN,내수배제,자성로140번길36,NaN,NaN
...,...,...,...,...,...,...,...,...,...
57,사상구,엄궁1,940.0,41089,90396,하천,강변대로 532번길 71,35.130758,128.970066
58,사상구,엄궁2,180.0,41089,90396,하천,강변대로 532번길 71,35.130758,128.970066
59,사상구,엄궁3,1700.0,41089,90396,하천,강변대로 532번길 71,35.130758,128.970066
60,사상구,학장,285.0,7440,16368,내수배제,농산물시장로 25번길 52-12,35.128991,128.967722


In [102]:
fig = px.scatter_mapbox(df_pump_re, lat="위도", lon="경도", hover_name="배수펌프장명", hover_data=["유수지 면적", "유수지 용량"],
                        color_discrete_sequence=["#FF6692"], zoom=11, height=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# 데이터 만들기 

In [103]:
df_pump_re.to_csv(path+'\유수지현황.csv', sep=',', na_rep='NaN',encoding='utf-8-sig')